In [ ]:
# Description

In [ ]:
import pandas as pd
import urllib.parse
import ipywidgets as widgets

In [ ]:
datasette_url = "https://datasette.planning.data.gov.uk/"

def get_spatial_dataset(dataset_combination):
    global spatial_dataset
    params_one = urllib.parse.urlencode({
        "sql": f"""
        select
            e.dataset,
            e.organisation_entity,
            e.reference
        from
            entity e
        """,
        "_size": "max"
    })
    params_two = urllib.parse.urlencode({
        "sql": f"""
        select
            e.dataset,
            e.organisation_entity,
            e.{dataset_combination[0]}
        from
            entity e
        """,
        "_size": "max"
    })
    url_one = f"{datasette_url}{dataset_combination[0]}.csv?{params_one}"
    df_one = pd.read_csv(url_one)
    url_two = f"{datasette_url}{dataset_combination[1]}.csv?{params_two}"
    df_two = pd.read_csv(url_two)

    df_one['reference'] = df_one['reference'].astype(str)
    df_two[f"{dataset_combination[0]}"] = df_two[f"{dataset_combination[0]}"].astype(str)
    
    #join
    df = pd.merge(df_one, df_two, left_on=["reference", "organisation_entity"], right_on=["reference", "organisation_entity"])
    
    return df

global collection_options    
collection_options = {
    "Article 4 Direction and Article 4 Direction Area": ["article-4-direction", "article-4-direction-area"],"Conservation Area and Documents": ["conservation-area","conservation-area-document"],"Tree Preservation Order and Tree": ["tree-preservation-order", "tree"], "Tree Preservation Order and Tree Preservation Zones":["tree-preservation-order", "tree-preservation-zone"]
}

collection_dropdown = widgets.Dropdown(
    options=collection_options,
    description="Select dataset combination:",
)

widgets.interact(get_spatial_dataset, dataset_combination=collection_dropdown)
initial_organisation = collection_dropdown.value